# Import and load csv

In [20]:
from transformers import pipeline
import pandas as pd

In [21]:
tweet_df = pd.read_csv("filtered_tweet_data.csv")
tweet_df.head()

,Unnamed: 0,post_date,body,like_num,ticker_symbol
0,0,2015-01-01,Jeff Bezos lost $7.4 billion in Amazon's worst...,57,AMZN
1,1,2015-01-01,"Earlier this month, a mysterious glitch caused...",17,AAPL
2,2,2015-01-01,Jeff Bezos lost $7.4 billion in Amazon's worst...,57,AMZN
3,3,2015-01-01,"New Post - ""Apple Stock Pullback: Price Target...",7,AAPL
4,4,2015-01-01,2015 technology forecasts: Wearable technology...,11,AAPL


# Classifier

In [19]:
classifier = pipeline("sentiment-analysis", model="ProsusAI/finbert")



Device set to use cpu


In [13]:
def get_finbert_sentiment(text):
    result = classifier(text)
    label = result[0]["label"]
    confidence = result[0]["score"]

    if label.lower() == "positive":
        sentiment = "bullish"
    elif label.lower() == "negative":
        sentiment = "bearish"
    else:
        sentiment = "neutral"


    return pd.Series([sentiment, confidence])

In [29]:
apple_df = tweet_df[tweet_df['ticker_symbol'] == 'AAPL']
apple_sample_df = apple_df.head(50)
apple_sample_df.head()

,Unnamed: 0,post_date,body,like_num,ticker_symbol
1,1,2015-01-01,"Earlier this month, a mysterious glitch caused...",17,AAPL
3,3,2015-01-01,"New Post - ""Apple Stock Pullback: Price Target...",7,AAPL
4,4,2015-01-01,2015 technology forecasts: Wearable technology...,11,AAPL
9,9,2015-01-01,"We're shocked, shocked! that Samsung might try...",9,AAPL
10,10,2015-01-01,"New hed, same story: The Apple press goes off ...",8,AAPL


In [ ]:
tweet_df[['sentiment_category', 'confidence_score']] = tweet_df['body'].apply(get_finbert_sentiment)
tweet_df.head()
tweet_df.to_csv("processed_tweets.csv")

